In [13]:
import pandas as pd
import torch

In [14]:
df = pd.read_csv('거제도.csv', encoding='cp949')
df

,지점,일시,평균 풍속(m/s),평균기압(hPa),평균 상대습도(%),평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,22104,2003-01-01,8.5,1023.8,52.0,2.9,15.9,2.0,0.7,1.0,3.3,7.6,9.5
1,22104,2003-01-02,5.6,1028.7,63.0,4.4,16.6,1.9,0.7,0.8,2.2,8.2,10.9
2,22104,2003-01-03,7.5,1018.6,64.0,5.2,16.9,1.8,0.6,0.8,3.1,9.2,11.8
3,22104,2003-01-04,10.3,1019.7,57.0,-0.6,16.7,2.1,0.7,0.9,2.8,12.5,75.0
4,22104,2003-01-05,8.8,1027.0,50.0,-2.6,15.8,2.0,0.7,0.8,2.6,9.7,10.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7053,22104,2022-12-27,6.7,1026.1,44.0,7.4,15.7,1.9,1.2,1.5,2.4,5.4,6.8
7054,22104,2022-12-28,6.7,1027.9,54.0,6.3,15.3,1.5,1.0,1.2,1.9,4.5,5.1
7055,22104,2022-12-29,6.8,1031.0,58.0,4.8,15.0,1.3,0.9,1.2,1.9,4.1,5.3
7056,22104,2022-12-30,5.4,1030.7,55.0,5.7,15.1,1.0,0.7,0.9,1.5,3.9,4.4


In [15]:
df = df.rename(columns={'일시': 'date', '평균 기온(°C)': 'Atemp(°C)', '평균 수온(°C)': 'Stemp(°C)', '평균 최대 파고(m)':'wave(m)','평균 풍속(m/s)':'Wind(m/s)','평균기압(hPa)':'Apress(hPa)','평균 상대습도(%)':'Moisture(%)'})
df=df.set_index(pd.to_datetime(df.date)) # Date를 인덱스로
df=df[['Wind(m/s)','Apress(hPa)','Moisture(%)','Atemp(°C)','Stemp(°C)','wave(m)']] # Column 재정리
df.dropna(subset=['Wind(m/s)','Apress(hPa)','Moisture(%)','Atemp(°C)','Stemp(°C)','wave(m)'], inplace=True)
df

,Wind(m/s),Apress(hPa),Moisture(%),Atemp(°C),Stemp(°C),wave(m)
date,,,,,,
2003-01-01,8.5,1023.8,52.0,2.9,15.9,2.0
2003-01-02,5.6,1028.7,63.0,4.4,16.6,1.9
2003-01-03,7.5,1018.6,64.0,5.2,16.9,1.8
2003-01-04,10.3,1019.7,57.0,-0.6,16.7,2.1
2003-01-05,8.8,1027.0,50.0,-2.6,15.8,2.0
...,...,...,...,...,...,...
2022-12-27,6.7,1026.1,44.0,7.4,15.7,1.9
2022-12-28,6.7,1027.9,54.0,6.3,15.3,1.5
2022-12-29,6.8,1031.0,58.0,4.8,15.0,1.3


In [16]:
df = df.loc['2013-01-01':]
df

,Wind(m/s),Apress(hPa),Moisture(%),Atemp(°C),Stemp(°C),wave(m)
date,,,,,,
2013-01-01,7.4,1020.2,58.0,5.5,13.2,1.7
2013-01-02,8.3,1026.2,54.0,3.8,14.2,1.8
2013-01-03,11.0,1032.3,54.0,-2.0,13.8,1.8
2013-01-04,6.8,1028.7,49.0,0.1,14.4,1.6
2013-01-05,6.6,1023.6,43.0,2.4,14.9,1.3
...,...,...,...,...,...,...
2022-12-27,6.7,1026.1,44.0,7.4,15.7,1.9
2022-12-28,6.7,1027.9,54.0,6.3,15.3,1.5
2022-12-29,6.8,1031.0,58.0,4.8,15.0,1.3


In [17]:
import pymysql
from sqlalchemy import create_engine, types

# Establish a connection to MySQL
host = 'localhost'
user = 'root'
password = 'k404'
database = 'geojedo'

df = df.reset_index()

conn = pymysql.connect(host=host, user=user, password=password, database=database)

# Create a SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

# Convert the DataFrame to a SQL table with specified column types
table_name = 'weather_data'
dtype = {'date': types.Date(), 'Wind (m/s)': types.Float(), 'Apress (hPa)': types.Float(), 'Moisture (%)': types.Float(), 'Atemp (°C)': types.Float(), 'Stemp (°C)': types.Float(), 'wave (m)': types.Float()}
df.to_sql(table_name, engine, if_exists='replace', index=False, dtype=dtype)

# Close the connection
conn.close()
